In [1]:
import polars as pl
import os
os.getcwd()


'c:\\Users\\digom\\Documents\\GitHub\\fh-energia'

In [41]:
df_cnpj = pl.scan_csv('./data/estabelecimentos.csv', has_header=False, infer_schema_length=0)
if False:
    import polars as pl
    # Assuming 'df' is your DataFrame
    n = 1  # Change this to the number of rows you want to drop

    df_cnpj = df_cnpj.slice(n, len(df_cnpj) - n)


In [ ]:
df_cnpj

In [42]:
df_cnpj = df_cnpj.select('column_1', 'column_2', 'column_3', 'column_4',  'column_12', 'column_14', 'column_15', 'column_16', 'column_19')
df_cnpj = df_cnpj.with_columns(pl.concat_str(
        [
            pl.col("column_1"),
            pl.col("column_2"),
            pl.col("column_3"),
        ],
        separator="",
    ).alias('cnpj'),
    pl.concat_str(
        [
            pl.col("column_14"),
            pl.col("column_15"),
            pl.col("column_16"),
        ],
        separator=" ",
    ).alias('endereco'))\
            .rename({'column_4': 'matriz_filial', 'column_19': 'CEP', 'column_12':'CNAE', 'column_16': 'numero'})\
            .select(['cnpj', 'endereco', 'matriz_filial', 'CEP','CNAE', 'numero'])


In [43]:
df_cnpj.head().collect()

cnpj,endereco,matriz_filial,CEP,CNAE,numero
str,str,str,str,str,str
"""012""","""13 14 15""","""3""","""18""","""11""","""15"""
"""07396865000168…","""RUA TUCANEIRA …","""1""","""89136000""","""1412602""","""30"""
"""64904295001851…","""AVENIDA MENINO…","""2""","""57046000""","""4639701""","""8551B"""
"""76016369000316…","""RUA DO COMERCI…","""2""","""11010141""","""4632001""","""55"""
"""52302726000182…","""RUA GREGORIO L…","""1""","""13472080""","""4712100""","""496"""


In [30]:
df_bdgd = pl.scan_csv('data/bdgd/ucmt_pj.csv', separator=";", encoding='utf8-lossy', infer_schema_length=0)
df_bdgd = df_bdgd.with_columns(pl.col('CEP').str.replace('\.', '').str.replace('-', '').alias('CEP'),
                               pl.col('CNAE').str.replace('-', '').str.replace('/', '').alias('CNAE'))

In [32]:

df_bdgd.select('CNAE').head().collect().to_pandas

<bound method DataFrame.to_pandas of shape: (5, 1)
┌─────────┐
│ CNAE    │
│ ---     │
│ str     │
╞═════════╡
│ 6462000 │
│ 1813099 │
│ 3600601 │
│ 3600601 │
│ 7490199 │
└─────────┘>

In [44]:
df_final = df_cnpj.join(df_bdgd, on=['CEP', 'CNAE'], how='inner')
df_final.head().collect()

cnpj,endereco,matriz_filial,CEP,CNAE,numero,COD_ID,DIST,PN_CON,PAC,CTMT,UNI_TR_AT,SUB,CONJ,MUN,CEG_GD,LGRD,BRR,CLAS_SUB,TIP_CC,FAS_CON,GRU_TEN,TEN_FORN,GRU_TAR,SIT_ATIV,DAT_CON,CAR_INST,LIV,ARE_LOC,TIP_SIST,DEM_CONT,DEM_01,DEM_02,DEM_03,DEM_04,DEM_05,DEM_06,…,ENE_05,ENE_06,ENE_07,ENE_08,ENE_09,ENE_10,ENE_11,ENE_12,DIC_01,DIC_02,DIC_03,DIC_04,DIC_05,DIC_06,DIC_07,DIC_08,DIC_09,DIC_10,DIC_11,DIC_12,FIC_01,FIC_02,FIC_03,FIC_04,FIC_05,FIC_06,FIC_07,FIC_08,FIC_09,FIC_10,FIC_11,FIC_12,SEMRED,DESCR,DATA_BASE,POINT_X,POINT_Y
str,str,str,str,str,str,str,str,str,str,str,str,str,str,str,str,str,str,str,str,str,str,str,str,str,str,str,str,str,str,str,str,str,str,str,str,str,…,str,str,str,str,str,str,str,str,str,str,str,str,str,str,str,str,str,str,str,str,str,str,str,str,str,str,str,str,str,str,str,str,str,str,str,str,str
"""58142886000198…","""RUA XAVIER PIN…","""1""","""11015090""","""4721102""","""253""","""2036308753""","""2937""","""1106146""","""148392427""","""VNO32""","""VNO-ETR104""","""VNO""","""13557""","""3548500""",null,"""R XAVIER PINHE…","""VL MATIAS""","""CO1""","""MT-Tipo1""","""ABCN""","""MT""","""49""","""A4""","""AT""","""14/10/2015""","""100000""","""0""","""UB""","""RD_INTERLIG""","""80""","""88""","""81""","""82""","""74""","""90""","""82""",…,"""20559""","""18684""","""20227""","""19008""","""18068""","""22513""","""22654""","""21726""","""0""","""0""","""0""","""0""","""5.473""","""0""","""0""","""0""","""0""","""0""","""0""","""0""","""0""","""0""","""0""","""0""","""1""","""0""","""0""","""0""","""0""","""0""","""0""","""0""","""1""",null,"""31DEC2022:00:0…","""-46.32441144""","""-23.95030581"""
"""87990800000185…","""AVENIDA FLORES…","""1""","""94910003""","""8411600""","""2251""","""3085212143""","""396""","""4987747""","""2969157""","""CCB23""","""CCB-ETR202""","""CCB""","""16531""","""4303103""",null,"""AV GAL FLORES …","""VL CACHOEIRINH…","""PP3""","""MT-Tipo10""","""ABCN""","""MT""","""63""","""A4""","""AT""","""07/06/2014""","""150""","""0""","""UB""","""RD_INTERLIG""","""150""","""173""","""153""","""146""","""116""","""93""","""93""",…,"""15701""","""15796""","""17203""","""15368""","""14505""","""18031.505""","""20978.339""","""21677""","""0""","""0""","""0""","""0""","""0""","""0""","""0.0661""","""0""","""0.1138""","""0.0541""","""0""","""0""","""0""","""0""","""0""","""0""","""0""","""0""","""1""","""0""","""1""","""1""","""0""","""0""","""1""",null,"""31DEC2022:00:0…","""-51.09229469""","""-29.94752995"""
"""07397290000106…","""AVENIDA GIOVAN…","""1""","""09111340""","""4731800""","""1980""","""MTE0018847""","""390""","""1039460""","""34796742738212…","""CAP0108""","""DCAP-EQ-TR1""","""DCAP""","""12874""","""3547809""",null,"""R GIOVANNI BAT…","""VILA HOMERO TH…","""CO1""","""MT-Tipo02""","""ABC""","""MT""","""49""","""A4""","""AT""","""02/08/2021""","""0""","""0""","""UB""","""RD_INTERLIG""","""135""","""150""","""150""","""150""","""150""","""150""","""150""",…,"""23678.51""","""28646.04""","""28304.9""","""26103.7""","""25580.02""","""27202.55""","""29078.56""","""37617.65""","""0""","""0""","""0""","""0""","""0""","""0""","""0""","""0""","""0""","""0""","""0""","""0""","""0""","""0""","""0""","""0""","""0""","""0""","""0""","""0""","""0""","""0""","""0""","""0""","""0""",null,"""31DEC2022:00:0…","""-46.69939982""","""-23.65417898"""
"""07397290000106…","""AVENIDA GIOVAN…","""1""","""09111340""","""4731800""","""1980""","""MTE0014319""","""390""","""1041048""","""34804840738206…","""CAP0109""","""DCAP-EQ-TR4""","""DCAP""","""12874""","""3547809""",null,"""R GIOVANNI BAT…","""VILA HOMERO TH…","""CO1""","""MT-Tipo01""","""ABC""","""MT""","""49""","""A4""","""AT""","""20/10/2020""","""299.7""","""0""","""UB""","""RD_INTERLIG""","""132.9""","""0""","""0""","""0""","""0""","""0""","""0""",…,"""0""","""0""","""0""","""0""","""0""","""0""","""0""","""0""","""0""","""0""","""0""","""0""","""0""","""0""","""0""","""0""","""0""","""0""","""0""","""0""","""0""","""0""","""0""","""0""","""0""","""0""","""0""

In [53]:
df_filter = df_final.filter(pl.col('LGRD').str.contains(pl.col('numero'), literal=True))

In [54]:
df_filter.head().collect()

cnpj,endereco,matriz_filial,CEP,CNAE,numero,COD_ID,DIST,PN_CON,PAC,CTMT,UNI_TR_AT,SUB,CONJ,MUN,CEG_GD,LGRD,BRR,CLAS_SUB,TIP_CC,FAS_CON,GRU_TEN,TEN_FORN,GRU_TAR,SIT_ATIV,DAT_CON,CAR_INST,LIV,ARE_LOC,TIP_SIST,DEM_CONT,DEM_01,DEM_02,DEM_03,DEM_04,DEM_05,DEM_06,…,ENE_05,ENE_06,ENE_07,ENE_08,ENE_09,ENE_10,ENE_11,ENE_12,DIC_01,DIC_02,DIC_03,DIC_04,DIC_05,DIC_06,DIC_07,DIC_08,DIC_09,DIC_10,DIC_11,DIC_12,FIC_01,FIC_02,FIC_03,FIC_04,FIC_05,FIC_06,FIC_07,FIC_08,FIC_09,FIC_10,FIC_11,FIC_12,SEMRED,DESCR,DATA_BASE,POINT_X,POINT_Y
str,str,str,str,str,str,str,str,str,str,str,str,str,str,str,str,str,str,str,str,str,str,str,str,str,str,str,str,str,str,str,str,str,str,str,str,str,…,str,str,str,str,str,str,str,str,str,str,str,str,str,str,str,str,str,str,str,str,str,str,str,str,str,str,str,str,str,str,str,str,str,str,str,str,str
"""58142886000198…","""RUA XAVIER PIN…","""1""","""11015090""","""4721102""","""253""","""2036308753""","""2937""","""1106146""","""148392427""","""VNO32""","""VNO-ETR104""","""VNO""","""13557""","""3548500""",null,"""R XAVIER PINHE…","""VL MATIAS""","""CO1""","""MT-Tipo1""","""ABCN""","""MT""","""49""","""A4""","""AT""","""14/10/2015""","""100000""","""0""","""UB""","""RD_INTERLIG""","""80""","""88""","""81""","""82""","""74""","""90""","""82""",…,"""20559""","""18684""","""20227""","""19008""","""18068""","""22513""","""22654""","""21726""","""0""","""0""","""0""","""0""","""5.473""","""0""","""0""","""0""","""0""","""0""","""0""","""0""","""0""","""0""","""0""","""0""","""1""","""0""","""0""","""0""","""0""","""0""","""0""","""0""","""1""",null,"""31DEC2022:00:0…","""-46.32441144""","""-23.95030581"""
"""01525323000676…","""AVENIDA SAO PA…","""2""","""87013313""","""5611203""","""743""","""86276697""","""2866""","""841516052""","""841182159""","""851480018""","""580391645""","""72760""","""15933""","""4115200""",null,"""SAO PAULO 743""","""ZONA 09 ARMAZE…","""CO9""","""MT-Tipo02""","""ABC""","""MT""","""49""","""A4""","""AT""","""03/09/2010""","""82""","""1""","""UB""","""RD_INTERLIG""","""82""","""76""","""74""","""74""","""72""","""66""","""66""",…,"""0""","""0""","""0""","""23720""","""21653.911""","""24819.096""","""23058.306""","""24674.872""","""0""","""0""","""0""","""0""","""0""","""0""","""0""","""0""","""0""","""0""","""0""","""0""","""0""","""0""","""0""","""0""","""0""","""0""","""0""","""0""","""0""","""0""","""0""","""0""","""0""",null,"""31DEC2022:00:0…","""-51.93320932""","""-23.41905605"""
"""01525323000676…","""AVENIDA SAO PA…","""2""","""87013313""","""5611203""","""743""","""DP_86276697""","""2866""","""841516052""","""841182159""","""851480018""","""580391645""","""72760""","""15933""","""4115200""",null,"""SAO PAULO 743""","""ZONA 09 ARMAZE…","""CO9""","""MT-Tipo02""","""ABC""","""MT""","""49""","""A4""","""DS""","""03/09/2010""","""82""","""0""","""UB""","""RD_INTERLIG""","""82""","""76""","""74""","""74""","""72""","""66""","""66""",…,"""24819.096""","""24819.096""","""24819.096""","""0""","""0""","""0""","""0""","""0""","""0""","""0""","""0""","""0""","""0""","""0""","""0""","""0""","""0""","""0""","""0""","""0""","""0""","""0""","""0""","""0""","""0""","""0""","""0""","""0""","""0""","""0""","""0""","""0""","""0""",null,"""31DEC2022:00:0…","""-51.93320932""","""-23.41905605"""
"""01525323000676…","""AVENIDA SAO PA…","""2""","""87013313""","""5611203""","""743""","""104471816""","""2866""","""584761614""","""584761895""","""851480018""","""580391645""","""72760""","""15933""","""4115200""",null,"""SAO PAULO 743""","""ZONA 09 ARMAZE…","""CO9""","""MT-Tipo06""","""ABC""","""MT""","""49""","""A4""","""AT""","""17/12/2018""","""90""","""1""","""UB""","""RD_INTERLIG""","""90""","""72""","""79""","""71""","""81""","""64""","""74""",…,"""18774.567""","""18405.387""","""20376.594""","""20600""","""20182.806""","""24251.787""","""23319.009""","""26757.108""","""0""","""0""","""0""","""0""","""0""","""0""","""0""","""0""","""0""","""0""","""0""","""0""","""0""","""0""","""0""","""0""","""0""","""0""","""

In [55]:
df_filter.sink_csv('./data/energia_com_cnpj.csv')